In [ ]:
# import the necessary libraries
import numpy as np
import time
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import cv2
import tensorflow as tf
import keras
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Dense, Dropout, Flatten


In [ ]:
def load_images(file, images_name, images, labels, crop=True):
    # file: a text file that has a name of an image and a label
    # images_name, images, labels: lists 
    with open(file) as f:
        for line in f:
            image_dir = line.strip("\n").split(":")[0]
            if "XY" in line:
                label = 0
                image_dir = image_dir
            elif "XZ" in line:
                label = 1
                image_dir = f"Processed_Images_XZ\{image_dir}"
            elif "YZ" in line:
                label = 2
                image_dir = f"Processed_Images_YZ\{image_dir}"
                
            # read the image
            image = cv2.imread(f"{image_dir}")
            # resize to 500 x 500
            if crop:
                image = cv2.resize(image, (500, 500))
            # from BGR to gray
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # append image and label to the list
            images_name.append(image_dir)
            images.append(image)
            labels.append(label)



In [ ]:
images_name, images, labels = [], [], []

In [ ]:
load_images("XY_labels.txt", images_name, images, labels)
load_images("XZ_labels.txt", images_name, images, labels)
load_images("YZ_labels.txt", images_name, images, labels)

In [ ]:
print(labels.count(2))

In [ ]:
# normalise and transform to np array
def normalise_images(images, labels):
    # Convert to numpy arrays
    images = np.array(images, dtype=np.float32)
    labels = np.array(labels)
    labels = labels.astype(np.int)

    # Normalise the images
    images /= 255.0

    return images, labels


In [ ]:
images_norm, labels = normalise_images(images, labels)


In [ ]:
# reshaping
images_norm = images_norm.reshape(-1,
                                  images_norm.shape[1], images_norm.shape[2], 1)
labels = to_categorical(labels, num_classes=len(np.unique(labels)))


In [ ]:
# 80-20 train test split
X_train, X_test, y_train, y_test = train_test_split(images_norm, labels, train_size=0.8, random_state=42)


In [ ]:
# building the CNN model
plane_model_light = Sequential()
plane_model_light.add(Conv2D(2, kernel_size=(3, 3), activation='linear',
                          input_shape=(X_train.shape[1], X_train.shape[2], 1), padding='same'))
plane_model_light.add(LeakyReLU(alpha=0.1))
plane_model_light.add(MaxPooling2D((2, 2), padding='same'))
plane_model_light.add(Conv2D(4, (3, 3), activation='linear', padding='same'))
plane_model_light.add(LeakyReLU(alpha=0.1))
plane_model_light.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
plane_model_light.add(Conv2D(4, (3, 3), activation='linear',
                          padding='same', name='just_do_it'))
plane_model_light.add(LeakyReLU(alpha=0.1))
plane_model_light.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
plane_model_light.add(Flatten())
plane_model_light.add(Dense(2, activation='linear'))
plane_model_light.add(LeakyReLU(alpha=0.1))
plane_model_light.add(Dense(y_train.shape[1], activation='softmax'))

plane_model_light.compile(loss=keras.losses.categorical_crossentropy,
                       optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])


In [ ]:
# saving untrained model for later use
plane_model_light.save("plane_model_light_untrained.h5")


In [ ]:

epochs = 20
batch_size = 64

In [ ]:
history = plane_model_light.fit(X_train, y_train, epochs=epochs,
                                   batch_size=batch_size, validation_split=0.1)


In [ ]:
def epoch_vs_performance_plot(history):
    # Plot the loss and accuracy curves for training and validation
    fig, ax = plt.subplots(2, 1)
    ax[0].plot(history.history['loss'], color='b', label="Training loss")
    ax[0].plot(history.history['val_loss'], color='r',
               label="validation loss", axes=ax[0])
    ax[1].set_xlabel("Number of epochs")
    legend = ax[0].legend(loc='best', shadow=True)

    ax[1].plot(history.history['accuracy'],
               color='b', label="Training accuracy")
    ax[1].plot(history.history['val_accuracy'],
               color='r', label="Validation accuracy")
    ax[1].set_xlabel("Number of epochs")
    legend = ax[1].legend(loc='best', shadow=True)


In [ ]:
def speed_and_acc(model, X_test, y_test):
    # show test accuracy and number of images processed per second
    start_time = time.time()
    test_loss, test_acc = model.evaluate(X_test, y_test)
    delta_time = time.time() - start_time
    num_img = X_test.shape[0]
    print("--- %s images per second ---" % (num_img/delta_time))
    print(f"test acc: {test_acc}")


In [ ]:
epoch_vs_performance_plot(history)


In [ ]:
# deciding on the number of epochs
epochs = 5

In [ ]:
plane_model_light = keras.models.load_model('plane_model_light_untrained.h5')


In [ ]:
history = plane_model_light.fit(X_train, y_train, epochs=epochs,
                                batch_size=batch_size, validation_split=0.1)


In [ ]:
speed_and_acc(plane_model_light, X_test, y_test)


In [ ]:
# Look at confusion matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')




In [ ]:
# Predict the values from the test set
Y_pred = plane_model_light.predict(X_test)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis=1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=["XY", "XZ", "YZ"])


In [ ]:
# loading untrained model
plane_model_light = keras.models.load_model('plane_model_light_untrained.h5')


In [ ]:
# training the model on the full dataset
plane_model_light.fit(images_norm, labels, epochs=epochs, batch_size=batch_size)


In [ ]:
# saving the final model
plane_model_light.save("plane_model_light_FINAL.h5")
